In [1]:
import nltk
import re
from nltk import FreqDist
from nltk import pos_tag
from nltk.corpus import wordnet as wn
from nltk.corpus import PlaintextCorpusReader
from nltk.tokenize import word_tokenize
from collections import defaultdict
from nltk.collocations import *
from nltk.corpus import stopwords
from stanfordcorenlp import StanfordCoreNLP
from nltk.tree import Tree
import time
from prettytable import PrettyTable

In [2]:
# Open the review file 
textfile = open("/Users/LXIN/Desktop/reviews.txt")
reviewText = textfile.read()
print(reviewText[:20])

This is a great tutu


In [3]:
# Tokenize
from nltk import tokenize
tokensen = tokenize.sent_tokenize(reviewText)
print(len(tokensen))
for s in tokensen[:50]:
	print(s)

1140642
This is a great tutu and at a really great price.
It doesn't look cheap at all.
I'm so glad I looked on Amazon and found such an affordable tutu that isn't made poorly.
A++
I bought this for my 4 yr old daughter for dance class, she wore it today for the first time and the teacher thought it was adorable.
I bought this to go with a light blue long sleeve leotard and was happy the colors matched up great.
Price was very good too since some of these go for over $15.00 dollars.
What can I say... my daughters have it in orange, black, white and pink and I am thinking to buy for they the fuccia one.
It is a very good way for exalt a dancer outfit: great colors, comfortable, looks great, easy to wear, durables and little girls love it.
I think it is a great buy for costumer and play too.
We bought several tutus at once, and they are got high reviews.
Sturdy and seemingly well-made.
The girls have been wearing them regularly, including out to play, and the tutus have stood up well.
Fi

In [ ]:
# Extract sentences 
startime = time.time()
nlp = StanfordCoreNLP('/Users/LXIN/Desktop/stanford-corenlp-full-2018-10-05', timeout = 15000)

In [ ]:
ques = []
im = []
imperative = []

In [ ]:
# Fuunctions for extracting 
def filt(x):
	return x.label()=='SQ' or x.label()=='SBARQ'
def f2(x):
	return x.label() == 'JJ'

In [ ]:
# Extract question sentences  
for sentence in tokensen:	
	if sentence[len(sentence)-1] == '?':
		a = (nlp.parse(sentence))
		parsetree = Tree.fromstring(a)
		for subtree in parsetree.subtrees(filter =  filt):
			for subtree in parsetree.subtrees(filter =  f2):
				ques.append(sentence)
				break
			break
		# Store the rest sentence in a new list for extracting imperative sentence later
	elif sentence[len(sentence)-1] == '!':
		im.append(sentence)
# First few question sentences 
print('Length of question sentences: ',len(ques))
for s in ques[:30]:
	print(s)
print('\n-----------------------------\n')
print('\nTime: ',(time.time()-startime))

nlp.close()

In [ ]:
# Extract imperative sentences 
startime = time.time()
im2 = []

rmVerb = ['Love','Will','Recommend','Want','Wonder','Seems']
rmVerbLow = ['love','will','recommend','want','wonder','seems']
titleWord = ['Always', 'Never', 'Please', 'Just']

# Include only the sentence with adj phrases 
for s in im:  
	tokenword = nltk.word_tokenize(s)  
	taggedtextStanford = nltk.pos_tag(tokenword)
	for word in taggedtextStanford:
		if word[1] == 'JJ':    
			im2.append(s)
			break
			
print(len(im2))
print(im2[:30])
print('\n-----------------------\n')

In [ ]:
# Extract Imperative sentences 
for s in im2:
	tokenword = nltk.word_tokenize(s)  
	taggedtextStanford = nltk.pos_tag(tokenword)
	if taggedtextStanford[0][0] in titleWord:
		if taggedtextStanford[1][1] == 'VB' and taggedtextStanford[1][0] not in rmVerbLow:
			imperative.append(s)
	if taggedtextStanford[0][1] == 'VB':
		if taggedtextStanford[0][0] not in rmVerb: 
			imperative.append(s)

print('Length of imperative sentences: ',len(imperative))
for s in imperative[:100]:
	print(s)
print('\nTime: ',(time.time()-startime))

In [ ]:
# Print the table 
lengthsQ = [len(i) for i in ques]
avgQ = sum(lengthsQ) / len(ques)

lengthsI = [len(i) for i in imperative]
avgI = sum(lengthsI) / len(imperative)

t = PrettyTable(['Type', 'Number of sentences', 'Average length of sentence'])
t.add_row(['Question sentence', len(ques), avgQ])
t.add_row(['Imperative sentence', len(imperative), avgI])
print(t)

In [ ]:
# Question Sentence Analysis 
# Question Sentences -- Unigram analysis
tokenQues = []
for s in ques:
	tokenQues.append(nltk.word_tokenize(s))

# Make a flat list out of list of lists(tokenQues)
tokenQues2 = [item for sublist in tokenQues for item in sublist]
# Convert all the alphabetical characters to lower case
alphaWords = [w.lower() for w in tokenQues2 if w.isalpha()]
print(alphaWords[:20])
# Remove stop words for unigram
stopwords = nltk.corpus.stopwords.words('english')
rmStop = [w for w in alphaWords if not w in stopwords]
# Lemmatization
tag_map = defaultdict(lambda: wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
wnl = nltk.WordNetLemmatizer()
lem = []
for rmStop, tag in pos_tag(rmStop):
	lemma = wnl.lemmatize(rmStop, tag_map[tag[0]])
	lem.append(lemma)
print(lem[:20])
print('\n-------------------------\n')
# Frequency distribution
fdist = FreqDist(lem)
fdistKey = list(fdist.keys())
topkey = fdist.most_common(50)
for pair in topkey:
	print(pair)
print('\n-------------------------\n')

In [ ]:
# Question Sentences -- Bigram analysis
from nltk.collocations import *
def alpha_filter(w):
	pattern = re.compile('^[^a-z]+$')
	if(pattern.match(w)):
		return True
	else:
		return False
	
lowercase = [w.lower() for w in tokenQues2] 
print('\nBigram-------------------------')
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(lowercase)
scored = finder.score_ngrams(bigram_measures.raw_freq)
first = scored[0]
for bscore in scored[:20]:
	print(bscore)

In [ ]:
# alpha bigrams
print('\nAlpha Bigrams-------------------------')
finder.apply_word_filter(alpha_filter)
scored1 = finder.score_ngrams(bigram_measures.raw_freq)
for bscore in scored1[:50]:
	print(bscore)

print('\nStopword Bigrams-------------------------')  
# stopword bigrams  -> Bigrams Result
finder.apply_word_filter(lambda w: w in stopwords)
scored2 = finder.score_ngrams(bigram_measures.raw_freq)
for bscore in scored2[:50]:
	print(bscore)

In [ ]:
# Imperative Sentence Analysis 
# imperative Sentences -- Unigram analysis
tokenIm = []
for s in imperative:
	tokenIm.append(nltk.word_tokenize(s))
	
# Make a flat list out of list of lists(tokenQues)
tokenIm2 = [item for sublist in tokenIm for item in sublist]
# Convert all the alphabetical characters to lower case
alphaWords = [w.lower() for w in tokenIm2 if w.isalpha()]
print(alphaWords[:20])
# Remove stop words for unigram
stopwords = nltk.corpus.stopwords.words('english')
rmStop = [w for w in alphaWords if not w in stopwords]
# Lemmatization
tag_map = defaultdict(lambda: wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
wnl = nltk.WordNetLemmatizer()
lem = []
for rmStop, tag in pos_tag(rmStop):
	lemma = wnl.lemmatize(rmStop, tag_map[tag[0]])
	lem.append(lemma)
print(lem[:20])
print('\n-------------------------\n')
# Frequency distribution
fdist = FreqDist(lem)
fdistKey = list(fdist.keys())
topkey = fdist.most_common(50)
for pair in topkey:
	print(pair)
print('\n-------------------------\n')
	

In [ ]:
# Question Sentences -- Bigram analysis
from nltk.collocations import *
def alpha_filter(w):
	pattern = re.compile('^[^a-z]+$')
	if(pattern.match(w)):
		return True
	else:
		return False
	
lowercase = [w.lower() for w in tokenIm2] 
print('\nBigram-------------------------')
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(lowercase)
scored = finder.score_ngrams(bigram_measures.raw_freq)
first = scored[0]
for bscore in scored[:20]:
	print(bscore)

In [ ]:
# alpha bigrams
print('\nAlpha Bigrams-------------------------')
finder.apply_word_filter(alpha_filter)
scored1 = finder.score_ngrams(bigram_measures.raw_freq)
for bscore in scored1[:50]:
	print(bscore)

print('\nStopword Bigrams-------------------------')  
# stopword bigrams  -> Bigrams Result
finder.apply_word_filter(lambda w: w in stopwords)
scored2 = finder.score_ngrams(bigram_measures.raw_freq)
for bscore in scored2[:50]:
	print(bscore)